<a href="https://colab.research.google.com/github/thnavear/Qualcomm-DL-Hackathon/blob/master/genai_hackathon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 109.7/109.7 kB 2.2 MB/s eta 0:00:00


In [ ]:
!git clone https://github.com/thnavear/Qualcomm-DL-Hackathon.git

In [ ]:
# prompt: 1.code to import "/content/test.csv"
# 2.use model "llama-3.3-70b-versatile" from groq to classify emails from test.csv (read columns "ticket_ID", "ticket_subject" and "ticket_body" from each row in "/content/test.csv" into the following attributes
# Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
# Priority: The urgency level such as low, medium, or high.
# Language: The language code for the email's language, such as en (English), de (German) etc. You can access the list of language codes from the following link.
# Type: The nature of the request, categorized as Incident, Request, Problem, or Change
# 3.Return your response as a JSON object with the following structure:
#    {
#        "Department": "...",
#        "Priority": "...",
#        "Language": "...",
#        "Type": "..."
#    }
# 4.Convert the JSON object into output.csv with columns "ticket_ID","department","type","priority","language"

import pandas as pd
import json
from groq import Groq
import time
#import backoff

# Install necessary libraries
#!pip install groq

# Initialize the Groq client
client = Groq(api_key="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")

try:
    df = pd.read_csv("/content/test.csv") # Correct file name
except FileNotFoundError:
    print("Error: test.csv not found. Please upload the file to /content/")
    exit()

results = []
for index, row in df.iterrows():
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']

    prompt = f"""Classify the following email into the given attributes.

Subject: {subject}
Body: {body}

Attributes:
Department: The responsible department (Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry)
Priority(Indicates the urgency and importance of the issue): low, medium, or high
Language(Indicates the language code in which the email is written): The language code (en, de, etc.).
Type: Incident, Request, Problem, or Change

Rules for classifying IT tickets into each Department:

1. **Technical Support**:
   - Issues related to hardware or software malfunctions.
   - Troubleshooting connectivity problems.
   - Assistance with system errors or crashes.

2. **Customer Service**:
   - General inquiries about services or products.
   - Requests for information or assistance not related to technical issues.
   - Feedback or complaints about service quality.

3. **Billing and Payments**:
   - Questions about invoices or billing statements.
   - Issues with payment processing or refunds.
   - Requests for payment plans or billing adjustments.

4. **Product Support**:
   - Inquiries about product features or usage.
   - Troubleshooting product-specific issues.
   - Requests for product updates or upgrades.

5. **IT Support**:
   - Internal IT issues within the organization.
   - Network or server maintenance requests.
   - User account management and access issues.

6. **Returns and Exchanges**:
   - Requests to return or exchange products.
   - Issues with product warranties or guarantees.
   - Inquiries about return policies and procedures.

7. **Sales and Pre-Sales**:
   - Questions about product pricing or availability.
   - Requests for product demonstrations or trials.
   - Inquiries about bulk purchases or special offers.

8. **Human Resources**:
   - Employee-related inquiries or issues.
   - Requests for HR policies or procedures.
   - Issues related to payroll or benefits.

9. **Service Outages and Maintenance**:
   - Reports of service outages or disruptions.
   - Scheduled maintenance notifications.
   - Requests for updates on ongoing maintenance activities.

10. **General Inquiry**:
    - Any inquiries that do not fit into the above categories.
    - Requests for information or assistance that are not specific to a particular department.

Rules for classifying IT tickets into each type:

1. **Incident**:
   - Any unplanned interruption to an IT service or reduction in the quality of an IT service.
   - Examples include system crashes, application errors, network outages, or hardware failures.
   - The primary goal is to restore normal service operation as quickly as possible.

2. **Request**:
   - Any formal request for something new or additional.
   - Examples include requests for new hardware or software, access to a system, or information.
   - These are typically routine and do not indicate a failure or issue with existing services.

3. **Problem**:
   - The underlying cause of one or more incidents.
   - Examples include recurring system crashes, frequent network outages, or persistent performance issues.
   - The focus is on identifying and resolving the root cause to prevent future incidents.

4. **Change**:
   - Any modification to an IT service or its components.
   - Examples include software updates, hardware upgrades, configuration changes, or new feature implementations.
   - Changes are typically planned and follow a structured process to minimize risk and impact.

Rules for classifying IT tickets into different priority levels:

1. **Low Priority**:
   - Issues that have minimal impact on operations and can be addressed during regular maintenance.
   - Examples include minor software bugs, cosmetic issues, or requests for non-essential features.
   - No immediate action is required, and the issue can be scheduled for resolution at a convenient time.

2. **Medium Priority**:
   - Issues that have a moderate impact on operations but do not halt critical functions.
   - Examples include performance degradation, intermittent connectivity issues, or requests for important but non-urgent changes.
   - These issues should be addressed in a timely manner but do not require immediate attention.

3. **High Priority**:
   - Issues that have a significant impact on operations and require immediate attention.
   - Examples include system outages, critical application failures, or security breaches.
   - These issues need to be resolved as quickly as possible to restore normal operations and minimize downtime.


Return the output in CSV format with the attributes: "ticket_ID", "department", "type", "priority", "language".
Pls DO NOT explain and give the reasoning behind the reponse."""

    completion = client.chat.completions.create(
        model="gemma2-9b-it",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_completion_tokens=200,
        top_p=1,
        stream=False
    )
    time.sleep(2.5)

    response = completion.choices[0].message.content
    print(index)
    print(response)
    '''response_str = response.strip()
    response_str = response.replace("```","")
    response_str = response_str.replace("json","")
    response_str = response_str.replace("\n","")
    results.append(response_str)

processed_results = []
for index, result in enumerate(results):
    try:
        json_data = json.loads(result)
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],
            "department": json_data.get("Department", ""),
            "type": json_data.get("Type", ""),
            "priority": json_data.get("Priority", ""),
            "language": json_data.get("Language", "")
        })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for result {index + 1}: {e}")
        print(f"Problematic Result: {result}")
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],
            "department": "",
            "type": "",
            "priority": "",
            "language": ""
        })

output_df = pd.DataFrame(processed_results)
output_df.to_csv("output.csv", index=False)'''

0
ticket_ID,department,type,priority,language
1,Technical Support,Incident,high,it
1
ticket_ID,department,type,priority,language
,Billing and Payments,Request,medium,de 

2
ticket_ID,department,type,priority,language
1,Technical Support,Incident,medium,fr 

3
ticket_ID,department,type,priority,language
,Billing and Payments,Request,medium,it
4
ticket_ID,department,type,priority,language
1,Technical Support,Incident,medium,it
5
ticket_ID,department,type,priority,language
1,Technical Support,Incident,medium,it
6
ticket_ID,department,type,priority,language
1,IT Support,Incident,high,it
7
ticket_ID,department,type,priority,language
1,"Technical Support","Incident",high,pt 

8
ticket_ID,department,type,priority,language
1,IT Support,Request,medium,es 

9
ticket_ID,department,type,priority,language
1,IT Support,Incident,medium,it
10
ticket_ID,department,type,priority,language
,Technical Support,Incident,medium,it
11
ticket_ID,department,type,priority,language
,Technical Support,Incident,high

In [ ]:
processed_results = []
for index, result in enumerate(results):
    try:
        json_data = json.loads(result)
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],
            "department": json_data.get("Department", ""),
            "type": json_data.get("Type", ""),
            "priority": json_data.get("Priority", ""),
            "language": json_data.get("Language", "")
        })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for result {index + 1}: {e}")
        print(f"Problematic Result: {result}")
        processed_results.append({
            "ticket_ID": df.iloc[index]['ticket_ID'],
            "department": "",
            "type": "",
            "priority": "",
            "language": ""
        })

Error decoding JSON for result 1: Expecting value: line 1 column 1 (char 0)
Problematic Result: <think>Okay, so I need to classify this email into the given attributes. Let me start by reading the email carefully.The subject is "Urgente: problema con la funzione di condivisione dello schermo di Zoom." That's Italian. So the language is definitely Italian, which corresponds to the code "it".Looking at the body, the user is addressing customer support about a high-priority issue with Zoom's screen sharing function during critical webinars. They're using Zoom version 5.11.0 and the problem is disrupting their operations, needing immediate attention. They're asking for a quick resolution or to be directed to someone who can help.Now, the attributes to classify are Department, Priority, Language, and Type.Starting with Department: The email is about a technical issue with Zoom's functionality. So, it's related to technical support. The options are Technical Support, Customer Service, etc. S

In [ ]:
output_df = pd.DataFrame(processed_results)
output_df.to_csv("output.csv", index=False)

In [ ]:
print(results)
#print(processed_results)

[]


In [ ]:
# prompt: write a code to process data from "/content/output_results.txt" to extract attributes department, type, priority, and language for every email

import pandas as pd

try:
    with open("/content/output_results.txt", 'r') as file:
        file_content = file.readlines()
except FileNotFoundError:
    print("Error: output_results.txt not found.")
    exit()

# Assuming each line in the file corresponds to an email's attributes
processed_data = []
for line in file_content:
    try:
        attributes = line.strip().split('<think>') # Adjust the delimiter if needed
        if len(attributes) >= 4:  # Ensure enough attributes are present
          processed_data.append({
              "department": attributes[0],
              "type": attributes[1],
              "priority": attributes[2],
              "language": attributes[3]
          })
        else:
            print(f"Skipping line: Insufficient attributes: {line.strip()}")
    except Exception as e:
        print(f"Error processing line: {line.strip()}. Error: {e}")
        # Handle the error appropriately, e.g., skip the line or add default values


df = pd.DataFrame(processed_data)

df

,department,type,priority,language
0,',"Okay, so I need to classify this email into th...","Okay, so I need to classify this email into th...","Okay, so I need to classify this email into th..."


In [ ]:
# prompt: 1.code to import "/content/test.csv"
# 2.use model "llama-3.3-70b-versatile" from groq to classify emails from test.csv (read columns "ticket_ID", "ticket_subject" and "ticket_body" from each row in "/content/test.csv" into the following attributes
# Department: The responsible department, such as Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry
# Priority: The urgency level such as low, medium, or high.
# Language: The language code for the email's language, such as en (English), de (German) etc.
# Type: The nature of the request, categorized as Incident, Request, Problem, or Change
# 3.Return your response as a csv file with the following columns:
#   "ticket_ID","Department","Priority","Language","Type"

import pandas as pd
import json
from groq import Groq
import time

# Install necessary libraries if not already installed
# !pip install groq

# Initialize the Groq client (replace with your actual API key)
client = Groq(api_key="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")

try:
    df = pd.read_csv("/content/test1.csv")
except FileNotFoundError:
    print("Error: test.csv not found. Please upload the file to /content/")
    exit()

results = []
for index, row in df.iterrows():
    ticket_id = row['ticket_ID']
    subject = row['ticket_subject']
    body = row['ticket_body']

    prompt = f"""Classify the following email into the given attributes.

Subject: {subject}
Body: {body}

Attributes:
Department: The responsible department (Technical Support, Customer Service, Billing and Payments, Product Support, IT Support, Returns and Exchanges, Sales and Pre-Sales, Human Resources, Service Outages and Maintenance, or General Inquiry)
Priority: The urgency level (low, medium, or high).
Language: The language code (en, de, etc.).
Type: The nature of the request (Incident, Request, Problem, or Change)

Return your response as a JSON object with keys "Department", "Priority", "Language", and "Type"."""

    completion = client.chat.completions.create(
        model="llama-3.3-70b-versatile", # Use the specified model
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2,
        max_completion_tokens=200,
        top_p=1,
        stream=False
    )

    response = completion.choices[0].message.content
    results.append({"ticket_ID": ticket_id, "response": response})

processed_results = []
for result in results:
    ticket_id = result["ticket_ID"]
    response = result["response"]
    try:
        json_data = json.loads(response)
        processed_results.append({
            "ticket_ID": ticket_id,
            "Department": json_data.get("Department", ""),
            "Priority": json_data.get("Priority", ""),
            "Language": json_data.get("Language", ""),
            "Type": json_data.get("Type", "")
        })
    except json.JSONDecodeError as e:
        print(f"Error decoding JSON for ticket ID {ticket_id}: {e}")
        print(f"Problematic Response: {response}")
        processed_results.append({
            "ticket_ID": ticket_id,
            "Department": "",
            "Priority": "",
            "Language": "",
            "Type": ""
        })

output_df = pd.DataFrame(processed_results)
output_df.to_csv("output.csv", index=False)

Error decoding JSON for ticket ID 1: Expecting value: line 1 column 1 (char 0)
Problematic Response: ```json
{
  "Department": "Technical Support",
  "Priority": "high",
  "Language": "it",
  "Type": "Incident"
}
```
Error decoding JSON for ticket ID 2: Expecting value: line 1 column 1 (char 0)
Problematic Response: ```json
{
  "Department": "Billing and Payments",
  "Priority": "medium",
  "Language": "de",
  "Type": "Incident"
}
```
Error decoding JSON for ticket ID 3: Expecting value: line 1 column 1 (char 0)
Problematic Response: ```json
{
  "Department": "Technical Support",
  "Priority": "medium",
  "Language": "fr",
  "Type": "Problem"
}
```
Error decoding JSON for ticket ID 4: Expecting value: line 1 column 1 (char 0)
Problematic Response: ```json
{
  "Department": "Billing and Payments",
  "Priority": "medium",
  "Language": "it",
  "Type": "Request"
}
```
Error decoding JSON for ticket ID 5: Expecting value: line 1 column 1 (char 0)
Problematic Response: ```json
{
  "Departme